In [7]:
from models.flownetS import FlowNetS
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

from datasets import flying_chairs

import json

In [8]:
train_dataset = flying_chairs(split="train")
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = flying_chairs(split="val")
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [15]:
USE_GPU = True
num_class = 100
dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [10]:
def epe_loss(pred, label):
    loss = torch.norm(label - pred, p=2, dim=1).mean()
    return loss

In [11]:
*_, flow = train_dataset[0]
flow.size()

dummy = torch.zeros((2, 384, 512))
epe_loss(dummy, flow)

tensor(0.2222)

In [16]:
def train(model, dataloader, epochs, **kwargs):
    optimizer = optim.Adam(model.parameters(), **kwargs)  # learning rate
    for e in range(epochs):
        for i, (im1, im2, y) in enumerate(dataloader):
            
            x = torch.cat([im1, im2], dim=1)
            
            model.train()
            x = x.to(device)
            y = y.to(device)
            
            # concatenate
            pred = model(x)
            loss = epe_loss(pred, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print("loss:", loss.item())

torch.cuda.empty_cache()
            
model = FlowNetS()
model = model.to(device)
# train(model, train_loader, epochs=1, lr=0.0001)


In [17]:
from utils import HardwareManager
import utils
HardwareManager.dtype = torch.float32

utils.eval(val_loader, model)



RuntimeError: norm(): input dtype should be either floating point or complex dtypes. Got Long instead.